In [1]:
import sys
sys.path.insert(0, '../utils')

import os
import cv2
import numpy as np
from typing import Dict, Tuple
from skimage import io
from dataGenerator import *

%reload_ext autoreload
%autoreload 2

In [2]:
DATA_BANK_DIRECTORY

'/home/mark/DataBank/probav_data/'

In [ ]:
# Load dataset
imgAllSets = generateImageSet(isTrainData=True, NIR=True)

# Remove outliers
imgAllSets = removeImageWithOutlierPixels(imageSet=imgAllSets, threshold=60000, isTrainData=True)

# Upscale
imgAllSetsUpScaled = upsampleImages(imageSets=imgAllSets, scale=3)  # 128x128 -> 384x384

# Convert dataset to numpy array
output = imageSet2NumpyArrayHelper(imgAllSetsUpScaled, imgAllSets, isGrayScale=True, isNHWC=False)

# Get shifts and trim dataset based on shift threshold
# Read more about this shifts happening in LR Images here
# https://kelvins.esa.int/proba-v-super-resolution/data/


output = correctShifts(output, upsampleScale=1)



# Add key value element in output dictionary
output['upsampleScale'] = 3

# Create Patches
output = generatePatchDataset(output, useUpsample=True, patchSize=96,
                                  thresholdPatchesPerImgSet=20, thresholdClarityLR=0.7,
                                  thresholdClarityHR=0.85)


# Return a list of input outputs (maybe a 5D numpy array)
normArray = generateNormArray(dirList=output['names'])

Correcting shifts in imagesets        : 100%|█████████▉| 561/562 [05:48<00:00,  1.32it/s]